In [1]:
print("Hello World")

Hello World


# Practical Budget Pacing Algorithms, PID Controler and Simulation Test Bed for eBay Marketplace Sponsored Search

Recreation of the simulations and techniques used by Ebay in this paper http://papers.adkdd.org/2023/papers/adkdd23-nguyen-practical.pdf


This notebook adds:
- Adds a new PID control method and compares with the pacing algorithms used in the paper
- Simulates user-interaction data, as we don't have access to it. 



In [6]:
import pandas as pd


In [2]:
# Simulate 1440 minutes in a day
# Simulate bids for competing campaigns (3 campaigns with different budgets)
# Calculate Metrics for each campaign
# Use different pacing algorithms to spend budget
# Add PID to each campaign
# Build a nice UI


# Create dataframe that has
# Keyword, Item, Time, pCTR(probability of click through rate), bid




In [446]:
# Simulate 1440 minutes in a day, M campaigns competing for N keywords, pCTR, bid

# Define data
data = {
    # Each item is a book that represents a campaign
    'Item': ['Time Series Analysis: Forecasting and Control', 'Practical Statistics for Data Scientists', 'Designing Data-Intensive Applications'],
    # They are all bidding on the same keyword
    'Keyword': ['statistics book', 'statistics book', 'statistics book'],
    # Probability of click through rate (This is a guess as we don't have the actual data)
    'pCTR': [0.03, 0.04, 0.02],
    # Bid amount (this will be randomized around this central amount)
    'Bid': [0.5, 0.48, 0.55],
    'Budget': [5000,5000,2000],
    'Spend': [0.0,0.0,0.0],
}

# Create DataFrame
items_keywords_df = pd.DataFrame(data)

# Print DataFrame
display(items_keywords_df)

,Item,Keyword,pCTR,Bid,Budget,Spend
0,Time Series Analysis: Forecasting and Control,statistics book,0.03,0.50,5000,0.0
1,Practical Statistics for Data Scientists,statistics book,0.04,0.48,5000,0.0
2,Designing Data-Intensive Applications,statistics book,0.02,0.55,2000,0.0


In [ ]:
# Simple Simulation with no budget constraints

# Simulate 1440 minutes in a day
    # Could have peaks of traffic around the day
    # Could have multiple impressions
    
# Simulate a bid around a keyword
 # Bid amount is a random number around the central bid amount
 # The click through rate is a random number around the central pCTR
 
  
 # Business Metrics
  # Total Clicks
  # Budget Spend
  # CTR
  # Cost Per Click
  
# Pacing Evaluation Metrics
 # Pacing Error. Smoothness of spend over the day


In [332]:
# Simulate a bid around a keyword
   # This should will take modifiers from control algorithms
import numpy as np
def auction(df):
    """
    Gets df with bidding information and simulates an auction
    
    Returns winner and price paid
    
    """
    bid_dict = {}
    for index, id in df.iterrows():
        bid_dict[index] = id['Bid']- np.random.uniform(0.0, 0.2)
    winner = max(bid_dict, key=bid_dict.get)
    try:
        # price paid is the bid of the second highest bidder
        price_paid = sorted(bid_dict.values(), reverse=True)[1]
    except:
        # If there is only one bidder, the price paid is the bid of the winner
        price_paid = bid_dict[winner]
    return winner, price_paid

winner, price_paid = auction(items_keywords_df)

print(f"The winner is {items_keywords_df.loc[winner]['Item']}")
print(f"The price paid is {price_paid}")

{0: 0.44522564141361276, 1: 0.37873406042373675}
Winner is 0
Price paid is 0.37873406042373675
The winner is Time Series Analysis: Forecasting and Control
The price paid is 0.37873406042373675


In [333]:
items_keywords_df.loc[2]

KeyError: 2

In [447]:
# Simulate unlimited budget over 1440 minutes
# iterate over 1440 minutes (flat traffic)
# Save the winner and price paid
# Build a dataframe with the results

simulation_results = []
for minute in range(1440):
    # TODO(Raul): Add choice
    # Remove winner if Spend is greater than Budget
    #items_keywords_df = items_keywords_df[items_keywords_df['Spend'] < items_keywords_df['Budget']]
    
    # Choose who enters the bid (% to budget strategy)
    
    selection_df = items_keywords_df.copy()
    
    selection_df['Enter_bid'] = selection_df.apply(lambda x: decide_budget_proportional_choice(x['Budget'], x['Spend']), axis=1)
    selection_df = selection_df[selection_df['Enter_bid'] == 1]  
    
    # If there are no bidders, skip the minute and save as "No Bid" 
    
    if selection_df.shape[0] == 0:
        simulation_results.append({
            'Minute': minute,
            'Winner': 'No Bid',
            'Price Paid': 0,
            "pCTR": 0,
        })
            
    else:    
            
        winner, price_paid = auction(selection_df)
        result = {
            'Minute': minute,
            'Winner': items_keywords_df.loc[winner]['Item'],
            'Price Paid': price_paid,
            "pCTR": items_keywords_df.loc[winner]['pCTR'] + np.random.normal(-0.01, 0.01),
        }
        
        # Update Spend so far for the winner
        items_keywords_df.loc[winner, 'Spend'] = items_keywords_df.loc[winner, 'Spend'] + calc_total_spend(calc_clicks(result['pCTR']), price_paid)
        
    simulation_results.append(result)
    
simulation_results_df = pd.DataFrame(simulation_results)

{0: 0.43803356134107463, 1: 0.28067381399849567, 2: 0.4462548544825207}
Winner is 2
Price paid is 0.43803356134107463
{0: 0.33260715086234155, 1: 0.39265820908426985, 2: 0.5322273861899953}
Winner is 2
Price paid is 0.39265820908426985
{0: 0.4490458472517253, 1: 0.37397446872140744, 2: 0.46830729553100936}
Winner is 2
Price paid is 0.4490458472517253
{0: 0.4959840696472151, 1: 0.30387514808650107, 2: 0.4334752053348918}
Winner is 0
Price paid is 0.4334752053348918
{0: 0.4808162680612554, 1: 0.47477454202720026, 2: 0.4041775521548137}
Winner is 0
Price paid is 0.47477454202720026
{0: 0.36039346011521484, 1: 0.39122631310874534, 2: 0.4499797217091539}
Winner is 2
Price paid is 0.39122631310874534
{0: 0.47470903048798524, 1: 0.34265774763707474, 2: 0.49370412562337873}
Winner is 2
Price paid is 0.47470903048798524
{0: 0.4462362804763468, 1: 0.4535136858153023, 2: 0.43347948760559385}
Winner is 1
Price paid is 0.4462362804763468
{0: 0.3206906088473952, 1: 0.2904399463286843, 2: 0.440933919

In [448]:
simulation_results_df.groupby("Winner").agg({"Price Paid": "mean", "pCTR": "mean", "Minute": "count"})

,Price Paid,pCTR,Minute
Winner,,,
Designing Data-Intensive Applications,0.403225,0.009987,483
No Bid,0.000000,0.000000,209
Practical Statistics for Data Scientists,0.375762,0.029339,414
Time Series Analysis: Forecasting and Control,0.390779,0.020172,543


In [449]:

# Add number of clicks, total spend, CTR, CPC
def calc_clicks(pCTR, total_impressions = 1000):
    return total_impressions * pCTR

def calc_total_spend(clicks, price_paid):
    return clicks * price_paid

simulation_results_df['Clicks'] = simulation_results_df.apply(lambda x: calc_clicks(total_impressions=1000, pCTR=x['pCTR']), axis=1)
simulation_results_df['Total Spend'] = simulation_results_df.apply(lambda x: calc_total_spend(x['Clicks'], x['Price Paid']), axis=1)


simulation_results_df.groupby("Winner").agg({"Price Paid": "mean", "pCTR": "mean", "Minute": "count", "Clicks": "sum", "Total Spend": "sum"})

# add cost per click
simulation_results_df['Cost Per Click'] = simulation_results_df['Total Spend'] / simulation_results_df['Clicks']

simulation_results_df.groupby("Winner").agg({"Price Paid": "mean", "pCTR": "mean", "Minute": "count", "Clicks": "sum", "Total Spend": "sum", "Cost Per Click": "mean"})


# Cost per click are very similar for all campaigns
# Can introduce quality scores and impression charges?


,Price Paid,pCTR,Minute,Clicks,Total Spend,Cost Per Click
Winner,,,,,,
Designing Data-Intensive Applications,0.403225,0.009987,483,4823.488350,1936.667915,0.403225
No Bid,0.000000,0.000000,209,0.000000,0.000000,NaN
Practical Statistics for Data Scientists,0.375762,0.029339,414,12146.284051,4555.457905,0.375762
Time Series Analysis: Forecasting and Control,0.390779,0.020172,543,10953.480977,4256.166336,0.390779


In [445]:
items_keywords_df

,Item,Keyword,pCTR,Bid,Budget,Spend
0,Time Series Analysis: Forecasting and Control,statistics book,0.03,0.50,10000,5874.286151
1,Practical Statistics for Data Scientists,statistics book,0.04,0.48,5000,4284.231797
2,Designing Data-Intensive Applications,statistics book,0.02,0.55,2000,1860.603233


In [437]:
test_df = items_keywords_df.copy()

def decide_budget_proportional_choice(Budget, Spend) -> int:
    """
    Choose a campaign based on the proportion of budget spent
    """
    p = (Budget - Spend) / Budget
    return np.random.choice([0, 1], p=[1-p, p])

decide_budget_proportional_choice(1000, 500)

test_df['Enter_bid'] = test_df.apply(lambda x: decide_budget_proportional_choice(x['Budget'], x['Spend']), axis=1)
test_df[test_df['Enter_bid'] == 1]


,Item,Keyword,pCTR,Bid,Budget,Spend,Enter_bid
0,Time Series Analysis: Forecasting and Control,statistics book,0.03,0.50,10000,4131.759713,1
1,Practical Statistics for Data Scientists,statistics book,0.04,0.48,5000,3971.153331,0


In [438]:
test_df[test_df['Enter_bid'] == 1]

,Item,Keyword,pCTR,Bid,Budget,Spend,Enter_bid
0,Time Series Analysis: Forecasting and Control,statistics book,0.03,0.5,10000,4131.759713,1


In [ ]:
# Introduce budget constraints
# Introduce pacing algorithms


# Budget
# Remaining time left: minutes slots left
# Remaining Click opportunity: Depends on forecasting Model (SHOULD do this)

# Error: Budget rate error




In [ ]:
# Implement Budget Cap (stops bidding when budget is reached)
# Total Spend is recorded

0

In [ ]:
TODO:
    - Add Metrics and visualizations over minutes
    - Add a smoothness metric for pacing
    - Add PID